In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np

In [ ]:
os.makedirs('dataset')
import shutil
src="/kaggle/input/skin-disease-dataset/skin-disease-datasaet"
des="/kaggle/working/dataset"
#shutil.copytree(src="/kaggle/input/skin-diseases-image-dataset",dst="/kaggle/working/sim")
if os.path.exists(des):
        shutil.rmtree(des)
        shutil.copytree(src, des)

In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working/dataset")

In [ ]:
# loc="/kaggle/working/dataset/IMG_CLASSES"
# !pip install split_folders
# import splitfolders
# import os
# os.makedirs('output')
# os.makedirs('output/train')
# os.makedirs('output/val')
# os.makedirs('output/test')
# splitfolders.ratio(loc,output = "output",seed = 42,ratio = (0.80,.1,.1))

In [ ]:
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("float32")

In [ ]:
# from tensorflow.keras.preprocessing import image_dataset_from_directory
# train_dir="./output/train"
# test_dir="./output/test"
# val_dir="./output/val"
# train_data=image_dataset_from_directory(train_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=True,seed=42)
# test_data=image_dataset_from_directory(test_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=False,seed=42)
# val_data=image_dataset_from_directory(val_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=False,seed=42)

In [ ]:
# ### INCEPTION V3
# from tensorflow.keras.preprocessing import image_dataset_from_directory
# train_dir="./output/train"
# test_dir="./output/test"
# val_dir="./output/val"

# train_data=image_dataset_from_directory(train_dir,batch_size=32,image_size=(299,299),label_mode='categorical',shuffle=True,seed=42)
# test_data=image_dataset_from_directory(test_dir,batch_size=32,image_size=(299,299),label_mode='categorical',shuffle=False,seed=42)
# val_data=image_dataset_from_directory(val_dir,batch_size=32,image_size=(299,299),label_mode='categorical',shuffle=False,seed=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir="./dataset/train_set"
test_dir="./dataset/test_set"
val_dir="./dataset/test_set"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',      
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',     
    subset='validation',
    shuffle=False
)

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',      
    shuffle=False
)
# train_data=image_dataset_from_directory(train_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=True,seed=42)
# test_data=image_dataset_from_directory(test_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=False,seed=42)
# val_data=image_dataset_from_directory(val_dir,batch_size=32,image_size=(224,224),label_mode='categorical',shuffle=False,seed=42)

In [ ]:
images, labels = next(train_generator)
print(images.shape)  # (32, 224, 224, 3)
print(labels.shape) 

In [ ]:
class_names=train_generator.class_indices
class_count=train_generator.num_classes
print(class_count)
print(class_names)

In [ ]:
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD, Adamax
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import categorical_crossentropy, sparse_categorical_crossentropy, KLDivergence


def build_model(hp):
    # InceptionV3
    base_model = InceptionV3(include_top=False, 
                             weights="imagenet", 
                             input_shape=(224, 224, 3), 
                             pooling=hp.Choice("pooling", ["avg", "max"])
                            )
        
    
    base_model.trainable = False  # Or fine-tune later
    
    x = base_model.output
    x = BatchNormalization()(x)

    # Tune regularizers ON/OFF
    use_kernel_reg = hp.Boolean("use_kernel_regularizer")
    use_bias_reg = hp.Boolean("use_bias_regularizer")
    use_activity_reg = hp.Boolean("use_activity_regularizer")

    # Define regularizer values (only used if enabled)
    kernel_reg = regularizers.l2(hp.Float("kernel_l2", 1e-4, 1e-2, sampling="log")) if use_kernel_reg and use_activity_reg else None
    bias_reg = regularizers.l1(hp.Float("bias_l1", 1e-4, 1e-2, sampling="log")) if use_bias_reg else None
    activity_reg = regularizers.l1(hp.Float("activity_l1", 1e-4, 1e-2, sampling="log")) if use_kernel_reg and use_activity_reg else None

    # Tune number of dense layers
    for i in range(hp.Int("num_dense_layers", 1, 3)):
        units = hp.Int(f"dense_units_{i}", min_value=64, max_value=512, step=32)
        l2_val = hp.Float(f"l2_{i}", 1e-3, 1e-1, sampling="log")
        l1_val = hp.Float(f"l1_{i}", 1e-3, 1e-1, sampling="log")
        x = Dense(
            units,
            activation='relu',
            kernel_regularizer=kernel_reg,
            activity_regularizer=activity_reg
        )(x)
        if hp.Boolean(f"batchnorm_{i}"):
            x = BatchNormalization()(x)
        x = Dropout(hp.Float(f"dropout_{i}", 0.2, 0.5, step=0.1))(x)


    output = Dense(class_count, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)

    # Optimizer and learning rate
    optimizer_name = hp.Choice("optimizer", ["adam", "adamax","sgd"])
    lr = hp.Float("learning_rate", 1e-3, 1e-1, sampling="log")

    if optimizer_name == "adam":
        optimizer = Adam(learning_rate=lr)
    elif optimizer_name == "sgd":
        optimizer = SGD(learning_rate=lr)
    else:
        optimizer = Adamax(learning_rate=lr)
        
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

early_stop = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    ModelCheckpoint(filepath='best_model.h5',monitor='val_accuracy',save_best_only=True,save_weights_only=False,verbose=1),
    ReduceLROnPlateau(monitor='val_loss',factor=0.3,patience=3,min_lr=1e-6,verbose=1)
]

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=100,  # Increase for deeper tuning
    executions_per_trial=1,
    directory="kt_regularized",
    project_name="skin_disease_full_tune"
)

tuner.search(train_generator , validation_data=test_generator , epochs=20)


In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")


In [ ]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(test_generator, validation_data=test_generator, epochs=30, callbacks=[early_stop])


In [ ]:
# ## efficient_net_b2
# from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
# from tensorflow.keras import regularizers
# from tensorflow.keras.models import Model, load_model, Sequential
# from tensorflow.keras.callbacks import EarlyStopping
# model_name='EfficientNetB2'
# base_model=tf.keras.applications.EfficientNetB2(include_top=False, weights="imagenet",input_shape=(224,224,3), pooling='max') 
# x=base_model.output
# #x=layers.GlobalAvgPool2D(name = "pooling_layer")(x)
# x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
# x=Dense(256, kernel_regularizer = regularizers.l2(l2=0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dense(128, kernel_regularizer = regularizers.l2(0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dropout(rate=.45, seed=42)(x)        
# output=Dense(class_count, activation='softmax')(x)
# model=Model(inputs=base_model.input, outputs=output)
# model.compile(tf.keras.optimizers.SGD(.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
# from tensorflow.keras.models import Model
# from tensorflow.keras.applications import ResNet101V2
# from tensorflow.keras import regularizers
# from tensorflow.keras.callbacks import EarlyStopping

# # Load the base ResNet101V2 model with ImageNet weights
# base_model = ResNet101V2(include_top=False, weights="imagenet", input_shape=(224, 224, 3), pooling='max')

# # Add additional layers on top of the base model
# x = base_model.output
# x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
# x = Dense(256, kernel_regularizer=regularizers.l2(l=0.016), 
#           activity_regularizer=regularizers.l1(0.006), 
#           bias_regularizer=regularizers.l1(0.006), activation='relu')(x)
# x = Dense(128, kernel_regularizer=regularizers.l2(l=0.016), 
#           activity_regularizer=regularizers.l1(0.006), 
#           bias_regularizer=regularizers.l1(0.006), activation='relu')(x)
# x = Dropout(rate=0.45, seed=42)(x)
# output = Dense(class_count, activation='softmax')(x)

# # Define the complete model
# model = Model(inputs=base_model.input, outputs=output)

# # Compile the model with optimizer, loss function, and metrics
# model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Define early stopping criteria
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Assuming you have your training and validation data defined as train_data, train_labels, val_data, val_labels
# # Example:
# # train_data, train_labels = ...
# # val_data, val_labels = ...

# # Train the model with early stopping
# history = model.fit(train_data,
#                     epochs=100,  # Example number of epochs
#                     validation_data=val_data,
#                     callbacks=[early_stopping])

# # Optionally, you can save the trained model
# model.save('my_resnet_model.h5')

In [ ]:
# ### MOBILENET V3
# from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
# from tensorflow.keras import regularizers
# from tensorflow.keras.models import Model, load_model, Sequential

# model_name='MobileNetV3'
# base_model = tf.keras.applications.MobileNetV3Large(include_top=False, weights="imagenet", input_shape=(224, 224, 3), pooling='max')

# x=base_model.output
# #x=layers.GlobalAvgPool2D(name = "pooling_layer")(x)
# x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
# x=Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dense(128, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dropout(rate=.45, seed=42)(x)        
# output=Dense(class_count, activation='softmax')(x)
# model=Model(inputs=base_model.input, outputs=output)
# model.compile(tf.keras.optimizers.Adamax(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# ### Inception net v3 on 10 classes

# from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
# from tensorflow.keras import regularizers
# from tensorflow.keras.models import Model, load_model, Sequential
# from tensorflow.keras.callbacks import EarlyStopping

# model_name='InceptionV3'
# base_model = tf.keras.applications.InceptionV3(include_top=False, weights="imagenet", input_shape=(224, 224, 3), pooling='max')

# x=base_model.output
# #x=layers.GlobalAvgPool2D(name = "pooling_layer")(x)
# x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
# x=Dense(256, kernel_regularizer = regularizers.l2(0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dense(128, kernel_regularizer = regularizers.l2(0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dropout(rate=.45, seed=42)(x)
# output=Dense(class_count, activation='softmax')(x)
# model=Model(inputs=base_model.input, outputs=output)
# model.compile(tf.keras.optimizers.Adamax(.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

In [ ]:
model.summary()

In [ ]:
# # history=model.fit(train_data,epochs = 22,
# #                      validation_data = val_data)
# history = model.fit(train_data, epochs=23,
#                     validation_data=val_data,
                    
#                     callbacks=early_stop)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('test Accuracy:', test_acc)
print('test loss:', test_loss)

In [ ]:
pred_probs = model.predict(test_generator)
pred_classes = pred_probs.argmax(axis =1)
y_labels = test_generator.labels
class_names = list(test_generator.class_indices.keys())
# for image,label in test_generator.unbatch():
#     y_labels.append(label.numpy().argmax())

from sklearn.metrics import classification_report

print("Classification Report:\n")
print(classification_report(y_labels, pred_classes, target_names=class_names))

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import matplotlib.image as mping
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

  # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])
    ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
    plt.xticks(rotation=70, fontsize=text_size)
    plt.yticks(fontsize=text_size)
    threshold = (cm.max() + cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=text_size)
        else:
            plt.text(j, i, f"{cm[i, j]}",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=text_size)

  # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")

In [ ]:
make_confusion_matrix(y_labels,pred_classes,classes = class_names,figsize = (20,20))

In [ ]:
acc=history.history["accuracy"]
loss=history.history["loss"]
val_acc=history.history["val_accuracy"]
val_loss=history.history["val_loss"]
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')

In [ ]:
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')

## rock-curve

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Predict probabilities for each class on validation data
val_probs = model.predict(test_generator)
# Extract true labels
true_labels = np.concatenate([y for x, y in test_generator], axis=0)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(class_count):
    fpr[i], tpr[i], _ = roc_curve(true_labels[:, i], val_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i in range(class_count):
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for class %s' % (roc_auc[i], class_names[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import tensorflow_hub as hub

def predict_skin_disease(image_path, model):
    class_names = class_names
    Images= []
    
    try:
        img = cv2.imread(str(image_path))
        Images.append(cv2.resize(img, (224, 224)))
        image = np.array(Images)

    except Exception as e:
        return f"Error processing image: {e}"

    try:
        pred = model.predict(image)
        predicted_class_index = np.argmax(pred)
        predicted_class_name = class_names[predicted_class_index]
        return predicted_class_name
    except Exception as e:
        return f"Error during prediction: {e}"


model = tf.keras.models.load_model('')
result = predict_skin_disease("", model)
print(result)


In [ ]:
# model.save("IncepV3-8-Class.h5")
model.save_weights("IncepV3-8-Class.weights.h5")

In [ ]:

from IPython.display import FileLink
# FileLink('IncepV3-8-Class.h5')
FileLink('IncepV3-8-Class.weights.h5')
